In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time

In [2]:
driver = webdriver.Firefox()

url = 'https://www.fnde.gov.br/sigpcadm/sistema.pu?operation=localizar'

# query_str = '#consulta=tipoConsulta%3D2%26anoConcessao%3D2019%26programaId%3DC7%26ufId%3DBA'

driver.get(url)
time.sleep(5)

In [3]:
form_input = {
    'cbTipoConsulta': 'Repasse',          # Tipo_de_OPC
    'cbAno': '2019',                      # Ano Inicio Vigencia
    'cbPrograma': 'ALIMENTAÇÃO ESCOLAR',  # Programa/Projeto
    'cbUf': 'Bahia'                       # UF
}

In [4]:
for element_id, element_content in form_input.items():
    print(f'{element_id}: {element_content}')
    element = driver.find_element_by_id(element_id)
    element.send_keys(element_content)
    time.sleep(2)
    element.send_keys(Keys.TAB)

cbTipoConsulta: Repasse
cbAno: 2019
cbPrograma: ALIMENTAÇÃO ESCOLAR
cbUf: Bahia


In [5]:
time.sleep(2)
element_id = 'ext-gen218'  # botão pesquisar
element = driver.find_element_by_id(element_id)
ActionChains(driver).move_to_element(element).click().perform()

In [6]:
element_id = 'ext-gen240' # tabela com resultados (body)
element = driver.find_element_by_id(element_id)
# element.text
table_content = element.get_attribute('innerHTML')
row_content = table_content.split('Repasse')

In [7]:
# row_content[1]

In [8]:
href='javascript:abrirNoPconline("2575915")'
driver.execute_script(href)
alert_obj = driver.switch_to.alert
alert_obj.accept()

In [9]:
# driver.close()